# 01-07 : Sentiment Analysis

In [1]:
import pandas as pd
from tqdm.auto import tqdm
from pyabsa import ABSAInstruction, AspectTermExtraction, AspectPolarityClassification

/bin/sh: 1: nvidia-smi: not found


No CUDA GPU found in your device
[2023-08-08 05:23:14] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27




Try to downgrade transformers<=4.29.0.






## Data Load

In [2]:
# load the human feedback dataset
df_source = pd.read_parquet('../../data/interim/01-06_human_classified.parquet')

# show the data loaded
print(df_source.shape)
display(df_source.head(3))

(235, 17)


/usr/local/lib/python3.11/multiprocessing/pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


,id,created_at,review_rating,review_title,review_content,business_slug,chatbot_related,chatbot_evidence,chatbot_classification,chatbot_description,chatbot_suggestion,complaint_classification,complaint_service,complaint_description,complaint_suggestion,human_chatbot_classification,human_complaint_classification
0,3344640,2021-01-07 13:22:34,1,No option to speak to the agent on the custome...,Am not able to call vodacom to block my number...,vodacom,1,The customer mentioned that the Tobi bot canno...,Customer care assistance,The customer is complaining about not being ab...,Improve the chatbot's availability and provide...,Customer care,Vodacom,The customer is unable to call Vodacom to bloc...,Provide an option for customers to speak to an...,"limited functionality, unable to contact human...",blacklist
1,3347241,2021-01-10 11:32:59,1,"Airtime charged, but not credited to my phone",Bought Airtime online through the Vodacom App ...,vodacom,1,The customer mentioned trying to chat with TOB...,Customer service,The customer complained about being thrown out...,Improve the stability of the chatbot to preven...,Billing,Airtime,The customer bought airtime online but it was ...,Investigate the issue and credit the airtime t...,"technical error, unable to contact human agent",missing airtime
2,3353838,2021-01-15 11:32:11,1,Chatbot Tobi/ Voice Bundle,I am disappointed at how your service has beco...,vodacom,1,The complaint mentions the introduction of a c...,Limited functionality,The chatbot has made it impossible for custome...,Improve the chatbot's capabilities to handle a...,Service issue,Voice Bundle,Failed to load voice bundle but debited the cu...,Load the customer's voice bundle or reimburse ...,"limited functionality, unable to contact human...",voice bundle


## Functions

In [3]:
def classify_aspects(data:pd.DataFrame, id_column:str='id', text_column:str='text') -> pd.DataFrame:
    """Perform abstract based sentinment analysis on data."""
    quadruples = []

    # create the extractor
    quadruple_extractor = ABSAInstruction.ABSAGenerator('multilingual')

    # classify  tweet
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        # get the row data
        id = row[id_column]
        text = row[text_column]

        # classify the tweet
        classification = quadruple_extractor.predict(text=text, max_length=512)

        # get the quadruples
        for quadruple in classification['Quadruples']:
            quadruple['id'] = id
            quadruples.append(quadruple)

    # return the results
    return pd.DataFrame(quadruples)

# test the function
classify_aspects(
    data=df_source.head(3),
    id_column='id',
    text_column='review_content')

[2023-08-08 05:23:24] (2.3.1) Downloading checkpoint:multilingual 
[2023-08-08 05:23:24] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-08-08 05:23:24] (2.3.1) Checkpoint already downloaded, skip


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': 'Am not able to call vodacom to block my number and blacklist my lost phone, as there is no option to select when I call 082135, there is no option on  *135#, and the Tobi bot cannot help because it says its busy with an upgrade. I need to get get through to Vodacom or they need to call me on ********** ', 'Quadruples': [{'aspect': 'vodacom', 'polarity': 'negative', 'opinion': 'NULL', 'category': 'SUPPORT#GENERAL'}]}
{'text': "Bought Airtime online through the Vodacom App and on the last step there was an error. The money has come off my credit card, but the airtime has not been credited to my phone. Tried the online chat and was thrown out ('your chat has been ended'), then tried calling the helpline and again did not get anywhere - told there were high caller volumes and I should chat tot TOBI the chatbot, or call back later and then was thrown out of the call. No option to just stay in a waiting line.... \nVery bad service Vodacom! ", 'Quadruples': [{'aspect': 'service Voda

,aspect,polarity,opinion,category,id
0,vodacom,negative,NULL,SUPPORT#GENERAL,3344640
1,service Vodacom,negative,bad,SUPPORT#GENERAL,3347241
2,service,negative,limited,SERVICE#GENERAL,3353838
3,chatbot,negative,NULL,SERVICE#GENERAL,3353838


## Classification

In [4]:
# add a new column combining the title and body
df_source['text'] = df_source['review_title'] + ' -- ' + df_source['review_content']
display(df_source.head(3))

,id,created_at,review_rating,review_title,review_content,business_slug,chatbot_related,chatbot_evidence,chatbot_classification,chatbot_description,chatbot_suggestion,complaint_classification,complaint_service,complaint_description,complaint_suggestion,human_chatbot_classification,human_complaint_classification,text
0,3344640,2021-01-07 13:22:34,1,No option to speak to the agent on the custome...,Am not able to call vodacom to block my number...,vodacom,1,The customer mentioned that the Tobi bot canno...,Customer care assistance,The customer is complaining about not being ab...,Improve the chatbot's availability and provide...,Customer care,Vodacom,The customer is unable to call Vodacom to bloc...,Provide an option for customers to speak to an...,"limited functionality, unable to contact human...",blacklist,No option to speak to the agent on the custome...
1,3347241,2021-01-10 11:32:59,1,"Airtime charged, but not credited to my phone",Bought Airtime online through the Vodacom App ...,vodacom,1,The customer mentioned trying to chat with TOB...,Customer service,The customer complained about being thrown out...,Improve the stability of the chatbot to preven...,Billing,Airtime,The customer bought airtime online but it was ...,Investigate the issue and credit the airtime t...,"technical error, unable to contact human agent",missing airtime,"Airtime charged, but not credited to my phone ..."
2,3353838,2021-01-15 11:32:11,1,Chatbot Tobi/ Voice Bundle,I am disappointed at how your service has beco...,vodacom,1,The complaint mentions the introduction of a c...,Limited functionality,The chatbot has made it impossible for custome...,Improve the chatbot's capabilities to handle a...,Service issue,Voice Bundle,Failed to load voice bundle but debited the cu...,Load the customer's voice bundle or reimburse ...,"limited functionality, unable to contact human...",voice bundle,Chatbot Tobi/ Voice Bundle -- I am disappointe...


In [5]:
# perform the classification
df_quadruples = classify_aspects(
    data=df_source,
    id_column='id',
    text_column='text')

# save the quadruples to disk
df_quadruples.to_parquet('../../data/interim/01-07_quadruples.parquet')

[2023-08-08 05:23:30] (2.3.1) Downloading checkpoint:multilingual 
[2023-08-08 05:23:30] (2.3.1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-08-08 05:23:30] (2.3.1) Checkpoint already downloaded, skip


  0%|          | 0/235 [00:00<?, ?it/s]

{'text': 'No option to speak to the agent on the customer care line  -- Am not able to call vodacom to block my number and blacklist my lost phone, as there is no option to select when I call 082135, there is no option on  *135#, and the Tobi bot cannot help because it says its busy with an upgrade. I need to get get through to Vodacom or they need to call me on ********** ', 'Quadruples': [{'aspect': 'customer care line', 'polarity': 'negative', 'opinion': 'NULL', 'category': 'SUPPORT#GENERAL'}]}
{'text': "Airtime charged, but not credited to my phone -- Bought Airtime online through the Vodacom App and on the last step there was an error. The money has come off my credit card, but the airtime has not been credited to my phone. Tried the online chat and was thrown out ('your chat has been ended'), then tried calling the helpline and again did not get anywhere - told there were high caller volumes and I should chat tot TOBI the chatbot, or call back later and then was thrown out of the